In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import zillow
import re
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from zillow import ValuationApi
pd.options.display.max_rows = 700\

import arcgis
from arcgis.gis import GIS
gis = GIS()

### Instantiate the Zillow API
After installing Zillow API in your terminal you should be able to load the zillow library and the valuation API.  Notice that you must input your zillow api key before moving ahead.  The Zillow api takes your personal key as it's first parameter input and will not run without one.  According to the zillow ToS you must use your own personal key and cannot share with other members of your team/company.

In [2]:
from zillow import ValuationApi

#insert your API key from zillow here
zid = 'X1-ZWz18cqbc8wgsr_7j3qs'

api = zillow.ValuationApi()

Read in your Address csv Data.  This will be important later for utilization in the code.  If you change the name of your dataframe here you will have to change it in all of the code below

In [15]:
#rochester = pd.read_csv('./us/ny/city_of_rochester.csv')

In [3]:
df = pd.read_csv('./us/ny/broome.csv')

C:\Users\tbaca\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Cleaning Data
Below are the few steps taken to clean the data.  The first is a simple drop/drop na, removing non-essential columns and removing rows with NaN information (notably, 'STREET' or street names and 'NUMBERS' or house numbers)

In [4]:
#df['CITY'] = 'CALAVERAS'
# Manually input city name if mmissing in dataframe

In [5]:
df.drop(['DISTRICT','REGION','ID','HASH','UNIT'], axis = 1, inplace = True)
df['CITY'] = 'Broome'
df.dropna(inplace = True)
df['POSTCODE'] = df['POSTCODE'].astype(int)
df.reset_index(inplace = True)

##Only for specific area!##

This loop runs through our data to replace any 'NUMBER' values seperated in the '120-124' format.  This allows us to search each individual property individually in the API. Be patient, this could take some time to complete.  Note the progress bar, this is a personal preference to track the loops progress.  If this does not run for you, simply run 'pip install progressbar2' in your terminal

In [ ]:
# for each in df['NUMBER']:
#     if '-' in each:
#         temp=re.findall('(\d*)-(\d*).*',str(each))
#         print(temp)
#             for i in range(int(temp[0][0]),int(temp[0][1])+1):
#                 temp_df = pd.DataFrame(df.loc[la['NUMBER'] == str(each)])
#                 temp_df = pd.DataFrame(temp_df)
#                 temp_df['NUMBER'] = int(i)
#         df = pd.concat([df,temp_df], axis = 0)
#         df.reset_index(inplace = True, drop = True)
#     df = df.drop(rochester[la['NUMBER'] == each].index)

### Zillow Estimator
At this point we have a full list of addresses and postal codes in our dataframe which we can use to enter into the Zillow API.  This forloop allows for some flexibility in the requests, if Zillow has no estimate on a property the loop should add a "None" value to the list.

In [9]:
from progressbar import ProgressBar
bar = ProgressBar(left=True,right = True)

This progress bar is a personal choice and not needed to run the function, it does allow for some clear understanding at which stage your loop is at and the ETA for completion.  It comes with its own share of issues though, please use with caution.

In [10]:
def Zestimate(zipcode):
    zestimate_list = []
    zipdf = df[df['POSTCODE'].isin(zipcode)]


    for i in bar(range(len(zipdf))):
        try:
            address = str(zipdf['NUMBER'][i]+' '+zipdf['STREET'][i]+'.,'+zipdf['CITY'][i]+', FL')
            postal_code = str(int(zipdf['POSTCODE'][i]))
            data = api.GetSearchResults(zid,address,postal_code)
            if data != None:
                zestimate_list.append(data.zestiamte.amount)
            else:
                pass
        except:
            pass
    #Worth noting that many of the addresses do no have corresponding values in the zillow database.
    #The below print output shows how many the API missed.
    print('Missing Values:',len(zipdf)-len(zestimate_list),'Gathered:',"{0:.2f}".format((len(zestimate_list))/len(zipdf)*100),'%')
    
    # This is the print output, change the print out to your preference.
    zestimate_list = [x for x in zestimate_list if x is not None]
    zestimate_list = np.array(zestimate_list)
    statistics = stats.describe(zestimate_list)
    std = np.std(zestimate_list)
    median = np.median(zestimate_list)
    print('    ==========================================')
    print('                 For Area', str(zipcode) )
    print('    ==========================================')
    print()
    print('     |  Values  |:',statistics[0])
    print('     | Minimum  |:', statistics[1][0])
    print('     | Maximum  |:', statistics[1][1])
    print('     |  Median  |:', median)
    print('     |   Mean   |:', float("{0:.2f}".format(statistics[2])))
    print('     |   Std    |:', float("{0:.2f}".format(std)))
    print('     | Skewness |:', float("{0:.2f}".format(statistics[4])))
    print('     | Kurtosis |:', float("{0:.2f}".format(statistics[5])))
    plt.hist(zestimate_list, edgecolor = 'black');
    plt.xlabel('House Value')
    plt.ylabel('Frequency')
    plt.xticks(rotation='vertical')
    return 

In [11]:
# Choose a postalcode from here for the function:
df['POSTCODE'].value_counts().head(5)

13760    22095
13905    15393
13901    11964
13850    11182
13903    11044
Name: POSTCODE, dtype: int64

In [ ]:
Zestimate([13760,13905,13901,13850,13902])


  0% (66 of 60634) |                     | Elapsed Time: 0:00:03 ETA:   0:45:45

In [ ]:
# Common Error - if there is a value error # out of range, rerun the bar code from above
# Print Statements 
# Address Missing Data